# Cinemática Inversa para el Hexapodo
Este documento contiene el progreso de la Interfaz Gráfica de Usuario (GUI) construida en Python 3 para generar y visualizar los movimientos de un robot Hexapodo a partir de sus dimensiones y especificaciones del usuario.

Inicialmente utilizaremos el modulo **Tkinter**, el cual es una biblioteca gráfica **Tcl/Tk** de Python para realizar nuestra GUI. Para ello, debemos conocer ciertos conceptos básicos los cuales se ilustraran con el siguiente ejemplo:


In [1]:
import tkinter as tk

# Testing window that display the current version of Tkinter
# tk._test()

# Read the whole documentation at http://effbot.org/tkinterbook/tkinter-index.htm

# To initialize Tkinter, we create a Tk root widget (an ordinary window).
root = tk.Tk()
# We create a label widget as child to the root window.
theLabel = tk.Label(root, text="This is too easy")
# We call the pack method on this widget. This tells it to size itself to fit the given text, and make itself visible.
theLabel.pack()
# We display the window using the Tkinter event loop. The program will stay in the event loop until we close the window
root.mainloop()

Inicialmente, se importa el módulo Tkinter con el alias **tk** a tráves de la siguiente línea de código:

```python
import tkinter as tk
```

Toda GUI en Tkinter necesita de una ventana raíz o *root* donde visualizar los otros componentes hijos o _widgets_ para nuestra aplicación. En este ejemplo, creamos un elemento de tipo *label*, el cual puede mostrar tanto textos como iconos o alguna otra imagen. Para este caso, especificamos que es un texto y a su vez al componente madre al que pertenece, _root_.

Luego llamamos el método `.pack()` en nuestro elemento para autoajustar su tamaño en función del texto y hacerlo visible. Aún así, no se visualizará nada debido hasta no ejecutar el **ciclo de eventos Tkinter**, el cual se encarga de manejar cualquier acción del usuario con la interfaz (clic, teclear, mensajes, etc).

## Definición de los marcos de referencia y modelos cinemáticos

### Marco de referencia global
El marco de referencia global para está simulación estará ubicado en una de las esquinas de la simulación (o terreno), el cual denotaremos por la letra **G** en todas nuestras notaciones. Además, los ejes de este marco de referencia global seguirán la regla de la mano derecha. 

### Marco de referencia para el cuerpo (_Trunk_)
En el caso de la araña, se sitúa el origen de los ejes coordenados a la misma altura del panel inferior del robot, ubicandose en su centro geométrico. Se toma la denotación de los giros en los ejes **x - y - z** como **roll - pitch - yaw**, siendo estos relativos al marco de referencia global. En base a ello, las respectivas matrices de rotación se muestrán en las siguientes ecuaciones con sus respectivos ángulos:
 
<img src="img/roll-pitch-and-yaw-angles.png" width="300"/>

\begin{align}
Roll: R_x(\gamma) = 
\begin{bmatrix}
    1 & 0 & 0 \\
    0 & cos(\gamma) & -sin(\gamma) \\
    0 & sin(\gamma) & cos(\gamma) 
\end{bmatrix}
,\
Pitch: R_y(\beta) =
\begin{bmatrix}
    cos(\beta) & 0 & sin(\beta) \\
    0 & 1 & 0 \\
    -sin(\beta) & 0 & cos(\beta)
\end{bmatrix}
,\
Yaw: R_z(\alpha) =
\begin{bmatrix}
    cos(\alpha) & -sin(\alpha) & 0 \\
    sin(\alpha) & cos(\alpha) & 0 \\
    0 & 0 & 1 
\end{bmatrix}
\end{align}

### Definición de los paramétros Denavit - Hartenberg para la cinemática de una pata.
Basandonos en la convención Denavit - Hartenberg para describir la geometria de una cadena cinemática / manipulador robótico; los cuales se definen en la tabla de abajo, podemos obtener el modelo cinemático para una pata. Las notaciones para cada eslabón (o servo) están basadas en los insectos, siendo el primer servo (Une el cuerpo con la pata) denominado ***Coxa***, el segundo servo ***Femur*** y el último servo ***Tibia***.

| Parametro | Definición |
| :---: | :---: |
| $\theta_{\mathrm{i}}$ | Es el ángulo comprendido entre los ejes $x_{\mathrm{i - 1}}$ y $x_{\mathrm{i}}$, medido en torno al eje $z_{\mathrm{i - 1}}$, en el sentido de la regla de la mano derecha. |
| $\mathrm{d_i}$ | Es la desviación del eslabón, es decir, la distancia comprendida desde el eje $x_{\mathrm{i - 1}}$ al eje $x_{\mathrm{i}}$ a lo largo del eje $z_{\mathrm{i - 1}}$. |
| $\mathrm{a_i}$	| Es la longitud cinemática del eslabón, es decir, la distancia entre el eje $z_{\mathrm{i - 1}}$ y el $z_{\mathrm{i}}$ a lo largo del eje $x_{\mathrm{i}}$ |
| $\alpha_{\mathrm{i}}$ | Es el ángulo comprendido entre los ejes $z_{\mathrm{i - 1}}$ y $z_{\mathrm{i}}$, medido en torno al eje $x_{\mathrm{i}}$, en sentido de la regla de la mano derecha. |

La obtención de los parametros se realiza siguiendo los pasos de la metodologia en la figura siguiente, con la visualización de cada eje coordenado, rotaciones y la tabla resumiendo los valores / variables del modelo cinemático.

| Eslabón | $\theta_{\mathrm{i}}$ | $\mathrm{a_i}$ | $\mathrm{d_i}$ | $\alpha_{\mathrm{i}}$ |
| :---: | :---: | :---: | :---: | :---: |
| 1 | $\theta_1$ | $\mathrm{L_1}$ | $\mathrm{d_L}$ | $\pi/2$ |
| 2 | $\theta_2$ | $\mathrm{L_2}$ | 0 | 0 |
| 3 | $\theta_3$ | $\mathrm{L_3}$ | 0 | 0 |

In [17]:
# Ejemplo de Sympy
from sympy import *
init_printing(use_latex='mathjax')
x = symbols('x')
a = Integral(cos(x)*exp(x), x)
Eq(a, a.doit())

⌠                 x           x       
⎮  x             ℯ ⋅sin(x)   ℯ ⋅cos(x)
⎮ ℯ ⋅cos(x) dx = ───────── + ─────────
⌡                    2           2    

In [1]:
from sympy import *
init_printing(use_latex='mathjax')
# Simbolos generales para las matrices, ángulos y variables.
alpha, beta, gamma = symbols('alpha beta gamma')
g_k, r_c, theta_i, alpha_i, a_i, d_i = symbols('gamma_k r_B theta_i alpha_i a_i d_i')
# Simbolos especificos.
# Ángulos de los servos (variables)
t_1, t_2, t_3 = symbols('theta_1 theta_2 theta_3')
# Longitud de los eslabones de las patas.
L_1, L_2, L_3, d_L = symbols('L_1 L_2 L_3 d_L')
# Esta es la traslación desde el origen coordenadas globales hasta el centro del robot.
X_t, Y_t, Z_t = symbols('X_t Y_t Z_t')
# Coordenadas de origen de una pata
x_rf, y_rf, z_rf = symbols('x_rf y_rf z_rf')
x_rm, y_rm, z_rm = symbols('x_rm y_rm z_rm')
x_rr, y_rr, z_rr = symbols('x_rr y_rr z_rr')
x_lf, y_lf, z_lf = symbols('x_lf y_lf z_lf')
x_lm, y_lm, z_lm = symbols('x_lm y_lm z_lm')
x_lr, y_lr, z_lr = symbols('x_lr y_lr z_lr')

# Matrices de rotación alrededor de cada eje con sus variables
RotMatrX = rot_axis1(gamma).T
RotMatrY = rot_axis2(beta).T
RotMatrZ = rot_axis3(alpha).T
# Matriz de rotación completa (Roll -> Pitch -> Yaw) o "balance, cabeceo, guiñada"
RotMatrRPY = RotMatrZ*RotMatrY*RotMatrX
# Matriz homogénea de transformación de cuerpo a coordenadas globales (B -> G)
TransfMatrRPY = RotMatrRPY.col_insert(3, Matrix([X_t, Y_t, Z_t])).row_insert(3, Matrix([[0, 0, 0, 1]]))

# Matriz general homogénea de traslación / rotación del eje central de la araña a cada pata.
# Llamado tambien ángulo local de guiñada o "Yaw"
TransfMatrBP = Matrix([[cos(g_k), -sin(g_k), 0, r_c*cos(g_k)], 
                       [sin(g_k), cos(g_k), 0, r_c*sin(g_k)], 
                       [0, 0, 1, 0], 
                       [0, 0, 0, 1]])
# Matriz general homogénea de transformación del eslabón i al eslabón i-1. 
TransfMatrLink = Matrix([[cos(theta_i), -sin(theta_i)*cos(alpha_i), sin(theta_i)*sin(alpha_i), a_i*cos(theta_i)], 
                         [sin(theta_i), cos(theta_i)*cos(alpha_i), -cos(theta_i)*sin(alpha_i), a_i*sin(theta_i)], 
                         [0, sin(alpha_i), cos(alpha_i), d_i], 
                         [0, 0, 0, 1]])
# Matrices de transformación para cada eslabón en una pata. 
# En orden: Matrix de eslabón 1 -> 0, 2 -> 1, 3 -> 2
TransfMatrL1 = TransfMatrLink.subs([(theta_i, t_1), (alpha_i, pi/2), (d_i, d_L), (a_i, L_1)])
TransfMatrL2 = TransfMatrLink.subs([(theta_i, t_2), (alpha_i, 0), (d_i, 0), (a_i, L_2)])
TransfMatrL3 = TransfMatrLink.subs([(theta_i, t_3), (alpha_i, 0), (d_i, 0), (a_i, L_3)])

# Matriz de transformación total desde elemento final de la pata hasta el cuerpo (Eslabón 0).
TransfMatrPaw = simplify(TransfMatrL1*TransfMatrL2*TransfMatrL3)

# Matrices de transformación de pata al centro del cuerpo.
# Basicamente es multiplicar la matriz de transformación total (anterior) con la de transformación desde el eje central.
# Notación general para las patas: R - Derecha, L - Izquierda // F - Frontal, M - Media, R - Trasera.
# En dirección contrareloj, empezando por la izquierda: 1 - LF, 2 - LM, 3 - LR, 4 - RR, 5 - RM, 6 - RF 
TransfMatrLF = TransfMatrBP.subs(g_k, 2*pi/3)*TransfMatrPaw
TransfMatrLM = TransfMatrBP.subs(g_k, pi)*TransfMatrPaw
TransfMatrLR = TransfMatrBP.subs(g_k, 4*pi/3)*TransfMatrPaw
TransfMatrRR = TransfMatrBP.subs(g_k, 5*pi/3)*TransfMatrPaw
TransfMatrRM = TransfMatrBP.subs(g_k, 2*pi)*TransfMatrPaw
TransfMatrRF = TransfMatrBP.subs(g_k, pi/3)*TransfMatrPaw

# Matrices de transformación global para cada pata para obtener las coordenadas finales de cada una.
TransfMatrLF_G = TransfMatrRPY*TransfMatrLF*Matrix([0, 0, 0, 1])
TransfMatrLM_G = TransfMatrRPY*TransfMatrLM*Matrix([0, 0, 0, 1])
TransfMatrLR_G = TransfMatrRPY*TransfMatrLR*Matrix([0, 0, 0, 1])
TransfMatrRR_G = TransfMatrRPY*TransfMatrRR*Matrix([0, 0, 0, 1])
TransfMatrRM_G = TransfMatrRPY*TransfMatrRM*Matrix([0, 0, 0, 1])
TransfMatrRF_G = TransfMatrRPY*TransfMatrRF*Matrix([0, 0, 0, 1])

# Algoritmo para la Cinemática Inversa del robot
# Transformación Inversa - Global hasta cuerpo (G -> B)
TransfMatrRPY_IK = RotMatrRPY.T.col_insert(3, -RotMatrRPY.T*Matrix([X_t, Y_t, Z_t])).row_insert(3, Matrix([[0, 0, 0, 1]]))
# Esto es equivalente a la siguiente operación.
# TransfMatrRPY_IK = simplify(TransfMatrRPY**-1)

# Transformación Inversa - Cuerpo hasta la pata (Eslabón 0)
TransfMatrLF_IK = TransfMatrBP.subs(g_k, 2*pi/3)**-1
TransfMatrLM_IK = TransfMatrBP.subs(g_k, pi)**-1
TransfMatrLR_IK = TransfMatrBP.subs(g_k, 4*pi/3)**-1
TransfMatrRR_IK = TransfMatrBP.subs(g_k, 5*pi/3)**-1
TransfMatrRM_IK = TransfMatrBP.subs(g_k, 2*pi)**-1
TransfMatrRF_IK = TransfMatrBP.subs(g_k, pi/3)**-1

# Transformación Inversa - Global hasta la pata (Eslabón 0).
# Se multiplica cada transformación por una matriz cuyas variables seran las coordenadas del elemento final por pata.
TransfMatrLF_Coxa = simplify(TransfMatrRPY_IK*TransfMatrLF_IK*Matrix([x_lf, y_lf, z_lf, 1]))
TransfMatrLM_Coxa = simplify(TransfMatrRPY_IK*TransfMatrLM_IK*Matrix([x_lm, y_lm, z_lm, 1]))
TransfMatrLR_Coxa = simplify(TransfMatrRPY_IK*TransfMatrLR_IK*Matrix([x_lr, y_lr, z_lr, 1]))
TransfMatrRR_Coxa = simplify(TransfMatrRPY_IK*TransfMatrRR_IK*Matrix([x_rr, y_rr, z_rr, 1]))
TransfMatrRM_Coxa = simplify(TransfMatrRPY_IK*TransfMatrRM_IK*Matrix([x_rm, y_rm, z_rm, 1]))
TransfMatrRF_Coxa = simplify(TransfMatrRPY_IK*TransfMatrRF_IK*Matrix([x_rf, y_rf, z_rf, 1]))

In [2]:
# Pruebas de la cinemática directa
# TransfMatrRF_G[0].subs([(alpha, 0), (beta, 0), (gamma, 0), 
#                         (t_1, 0), (t_2, pi/2), (t_3, 0), 
#                         (L_1, 38.5), (L_2, 56.5), (L_3, 143.5), 
#                         (d_L, 45), (r_c, 137)])

# TransfMatrRM_G[0].subs([(alpha, 0), (beta, 0), (gamma, 0), 
#                         (t_1, 0), (t_2, 0), (t_3, pi/2), 
#                         (L_1, 38.5), (L_2, 56.5), (L_3, 143.5), 
#                         (d_L, 45), (r_c, 137)])
TransfMatrRF_Coxa

⎡                                                                             
⎢                                                                          -Xₜ
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢Xₜ⋅(sin(α)⋅cos(γ) - sin(β)⋅sin(γ)⋅cos(α)) - Yₜ⋅(sin(α)⋅sin(β)⋅sin(γ) + cos(α)
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢-Xₜ⋅(sin(α)⋅sin(γ) + sin(β)⋅cos(α)⋅cos(γ)) - Yₜ⋅(sin(α)⋅sin(β)⋅cos(γ) - sin(γ
⎢                                                                             
⎢                                                                             
⎣                                                   

### Solución de la cinemática inversa para cada pata de manera geométrica
Ya con la transformación de la coordenada global final para cada pata hasta el eslabón 0 (Unión de la pata con el cuerpo), procedemos a resolver de manera geometrica los ángulos de las patas necesarios para lograr dichas coordenadas. 

El ángulo $\theta_{\mathrm{1}}$ que debe formar el servo 1 (*Coxa*) es muy fácil de sacar con solo las coordenadas $x_k$ y $y_k$ del elemento final de la pata $k$. Aún así, hay que tener cuidado en el ángulo a calcular debido a los siguientes casos:

- La posición final se encuentra justo debajo de la articulación 1: Se generan múltiples soluciones debido a que la rotación de este servo no afectará la solución.
- La posición final se encuentra cerca del centro del robot: Puede generarse inversión del eje y por ende, se debe aplicar correciones para evitar dicha situación.

\begin{align}
\Theta_1(k) =  \begin{Bmatrix}
 \Theta_1(k-1) & ,x_L = 0 \\ 
 atan2(y_L,x_L) + 180^{\circ} & ,x_L < 0\\ 
 atan2(y_L,x_L) & ,otros
\end{Bmatrix}
\end{align}

Siendo $\Theta_1(k)$ el ángulo en el tiempo $k$, $x_L$ la componente en el eje x del efector final de la pata y $y_L$ su componente en el eje y.

Los ángulos $\Theta_2(k)$ y $\Theta_3(k)$ se puede encontrar a partir de la transformaciòn de las coordenadas finales de la pata desde el cuerpo hasta el sistema de coordenadas del *Coxa*, mediante la matriz de transformaciòn inversa $T_0^1 = (T_1^0)^{-1}$. Con ello, podemos obtener geometricamente las ecuaciones de los àngulos como se muestra en la siguiente figura:

<img src="img/IK-Leg-Top.jpg" width="300"/> <img src="img/IK-Leg-Side.jpg" width="300"/>

\begin{align}
\Theta_2(k) = acos \left ( \frac{(L_2)^2+b^2-(L_3)^2}{2*L_2*b} \right) + atan2\left ( z_c(k),x_c(k) \right )
\newline
\Theta_3(k) = 180^{\circ} - acos\left ( \frac{x_c(k)^{2} + z_c(k)^{2} - \left ( L_2\right )^{2} - \left ( L_3 \right )^{2}}{-2*L_3*L_2} \right )
\end{align}